In [3]:
# Байесовское A/B тестирование
from scipy.stats import beta
import numpy as np

from math import lgamma
from numba import jit

# Определение calc_prob_between функции
@jit
def h(a, b, c, d):
    num = lgamma(a + c) + lgamma(b + d) + lgamma(a + b) + lgamma(c + d)
    den = lgamma(a) + lgamma(b) + lgamma(c) + lgamma(d) + lgamma(a + b + c + d)
    return np.exp(num - den)

@jit
def g0(a, b, c):
    return np.exp(lgamma(a + b) + lgamma(a + c) - (lgamma(a + b + c) + lgamma(a)))

@jit
def hiter(a, b, c, d):
    while d > 1:
        d -= 1
        yield h(a, b, c, d) / d

def g(a, b, c, d):
    return g0(a, b, c) + sum(hiter(a, b, c, d))

def calc_prob_between(beta1, beta2):
    return g(beta1.args[0], beta1.args[1], beta2.args[0], beta2.args[1])

# Показы страницы и конверсия для контрольной и тестовой группы
imps_ctrl,convs_ctrl=16500, 30
imps_test, convs_test=17000, 50

# Инициализируются две beta функции
a_C, b_C = convs_ctrl+1, imps_ctrl-convs_ctrl+1
beta_C = beta(a_C, b_C)
a_T, b_T = convs_test+1, imps_test-convs_test+1
beta_T = beta(a_T, b_T)

# На сколько конверсия для тестовой группы выше, чем для контрольной
lift=(beta_T.mean()-beta_C.mean())/beta_C.mean()

# Рсчет вероятности для тестовой и контрольной групп
prob=calc_prob_between(beta_T, beta_C)

print (f"Test option lift Conversion Rates by {lift*100:2.2f}% with {prob*100:2.1f}% probability.")


Test option lift Conversion Rates by 59.68% with 98.2% probability.
